In [1]:
# Copyright 2018 The TensorFlow Authors.
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#https://github.com/tensorflow/docs/blob/master/site/en/tutorials/sequences/text_generation.ipynb

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
file='1400-0.txt'
url='https://www.gutenberg.org/files/1400/1400-0.txt' # Great Expectations by Charles Dickens


In [4]:
path = tf.keras.utils.get_file(file,url)

In [5]:
text = open(path).read()
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1013445 characters


In [6]:
# strip off text we don't need
text = text[835:]

# Take a look at the first 300 characters in text
print(text[:300])

My father's family name being Pirrip, and my Christian name Philip, my
infant tongue could make of both names nothing longer or more explicit
than Pip. So, I called myself Pip, and came to be called Pip.

I give Pirrip as my father's family name, on the authority of his
tombstone and my sister,--Mrs


In [7]:
# The unique characters in the file
vocabulary = sorted(set(text))
print ('{} unique characters.'.format(len(vocabulary)))

84 unique characters.


In [8]:
vocabulary_size = len(vocabulary)

In [9]:
# Creating a dictionary of unique characters to indices
char_to_index = {char:index for index, char in enumerate(vocabulary)}
print(char_to_index)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F': 33, 'G': 34, 'H': 35, 'I': 36, 'J': 37, 'K': 38, 'L': 39, 'M': 40, 'N': 41, 'O': 42, 'P': 43, 'Q': 44, 'R': 45, 'S': 46, 'T': 47, 'U': 48, 'V': 49, 'W': 50, 'X': 51, 'Y': 52, 'Z': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, 'ê': 80, 'ô': 81, '“': 82, '”': 83}


In [10]:
index_to_char = np.array(vocabulary)
print(index_to_char)
text_as_int = np.array([char_to_index[char] for char in text])

['\n' ' ' '!' '$' '%' '&' "'" '(' ')' '*' ',' '-' '.' '/' '0' '1' '2' '3'
 '4' '5' '6' '7' '8' '9' ':' ';' '?' '@' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r'
 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' 'ê' 'ô' '“' '”']


In [11]:
print('{')
for char,_ in zip(char_to_index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_to_index[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '%' :   4,
  '&' :   5,
  "'" :   6,
  '(' :   7,
  ')' :   8,
  '*' :   9,
  ',' :  10,
  '-' :  11,
  '.' :  12,
  '/' :  13,
  '0' :  14,
  '1' :  15,
  '2' :  16,
  '3' :  17,
  '4' :  18,
  '5' :  19,
  ...
}


In [12]:
# Show how the first 15 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:15]), text_as_int[:15]))

"My father's fam" ---- characters mapped to int ---- > [40 78  1 59 54 73 61 58 71  6 72  1 59 54 66]


In [13]:
# The maximum length sentence we want for a single input in characters
sequence_length = 100
examples_per_epoch = len(text)//sequence_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for char in char_dataset.take(5):
  print(index_to_char[char.numpy()])

M
y
 
f
a


In [14]:
sequences = char_dataset.batch(sequence_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index_to_char[item.numpy()])))

"My father's family name being Pirrip, and my Christian name Philip, my\ninfant tongue could make of bo"
'th names nothing longer or more explicit\nthan Pip. So, I called myself Pip, and came to be called Pip'
".\n\nI give Pirrip as my father's family name, on the authority of his\ntombstone and my sister,--Mrs. J"
'oe Gargery, who married the blacksmith.\nAs I never saw my father or my mother, and never saw any like'
'ness\nof either of them (for their days were long before the days of\nphotographs), my first fancies re'


In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(index_to_char[input_example.numpy()])))
  print ('Target data:', repr(''.join(index_to_char[target_example.numpy()])))

Input data:  "My father's family name being Pirrip, and my Christian name Philip, my\ninfant tongue could make of b"
Target data: "y father's family name being Pirrip, and my Christian name Philip, my\ninfant tongue could make of bo"


In [17]:
for char, (input_index, target_index) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(char))
    print("  input: {} ({:s})".format(input_index, repr(index_to_char[input_index])))
    print("  expected output: {} ({:s})".format(target_index, repr(index_to_char[target_index])))

Step    0
  input: 40 ('M')
  expected output: 78 ('y')
Step    1
  input: 78 ('y')
  expected output: 1 (' ')
Step    2
  input: 1 (' ')
  expected output: 59 ('f')
Step    3
  input: 59 ('f')
  expected output: 54 ('a')
Step    4
  input: 54 ('a')
  expected output: 73 ('t')


In [18]:

# Batch size
batch = 64
steps_per_epoch = examples_per_epoch//batch

# TF data maintains a buffer in memory in which to shuffle data
# since it is designed to work with possibly endless data
buffer = 10000

dataset = dataset.shuffle(buffer).batch(batch, drop_remainder=True)

dataset = dataset.repeat()

dataset

<RepeatDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [19]:
#  The vocabulary length in characterrs
vocabulary_length = len(vocabulary)

# The embedding dimension
embedding_dimension = 256

# Number of RNN units
recurrent_nn_units = 1024

In [20]:
if tf.test.is_gpu_available():
    recurrent_nn = tf.compat.v1.keras.layers.CuDNNGRU
    print("GPU in use")
else:
    import functools
    recurrent_nn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
    print("CPU in use")

CPU in use


In [21]:

def build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size):
    model = tf.keras.Sequential(
        [tf.keras.layers.Embedding(vocabulary_size, embedding_dimension, batch_input_shape=[batch_size, None]),
    recurrent_nn(recurrent_nn_units, return_sequences=True, recurrent_initializer='glorot_uniform', stateful=True),
    tf.keras.layers.Dense(vocabulary_length)
  ])
    return model

In [22]:
model = build_model(
  vocabulary_size = len(vocabulary),
  embedding_dimension=embedding_dimension,
  recurrent_nn_units=recurrent_nn_units,
  batch_size=batch)

In [23]:
for batch_input_example, batch_target_example in dataset.take(1):
    batch_predictions_example = model(batch_input_example)
    print(batch_predictions_example.shape, "# (batch, sequence_length, vocabulary_length)")

(64, 100, 84) # (batch, sequence_length, vocabulary_length)


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           21504     
_________________________________________________________________
unified_gru (UnifiedGRU)     (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 84)            86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(logits=batch_predictions_example[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
sampled_indices

array([61, 33, 47, 79, 37, 58, 25, 71, 28, 81, 24, 34,  9,  6, 83, 77, 18,
       57, 26,  5, 81, 56, 58, 23, 44, 58, 64, 39, 24,  9, 42, 21, 27, 38,
       74, 68, 53, 40,  5, 82,  3, 71, 14, 66, 60,  0,  4, 13, 16, 11, 20,
       44, 54, 32,  5,  3,  8, 13,  6, 52, 22, 66, 12, 77,  8, 23, 18, 55,
       26, 59, 38, 69, 12, 71, 45, 81, 12, 17, 36, 40, 40, 47, 40, 63, 40,
       40, 54, 60, 12, 62, 39, 15, 39, 74, 40, 20,  1, 26,  6, 25])

In [27]:
print("Input: \n", repr("".join(index_to_char[batch_input_example[0]])))

print("Next Char Predictions: \n", repr("".join(index_to_char[sampled_indices ])))
#

Input: 
 'r, that I might refer to it again; but I could not find it, and\nwas uneasy to think that it must hav'
Next Char Predictions: 
 "hFTzJe;rAô:G*'”x4d?&ôce9QekL:*O7@KuoZM&“$r0mg\n%/2-6QaE&$)/'Y8m.x)94b?fKp.rRô.3IMMTMjMMag.iL1LuM6 ?';"


In [28]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [29]:

batch_loss_example  = tf.compat.v1.losses.sparse_softmax_cross_entropy(batch_target_example, batch_predictions_example)
print("Prediction shape: ", batch_predictions_example.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", batch_loss_example.numpy())

Prediction shape:  (64, 100, 84)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4306927


In [30]:
#next produced by upgrade script.... 
#model.compile(optimizer = tf.compat.v1.train.AdamOptimizer(), loss = loss) 
#.... but following optimizer is available.
model.compile(optimizer = tf.optimizers.Adam(), loss = loss)

In [31]:
# Directory where the checkpoints will be saved
directory = './checkpoints'
# Name of the checkpoint files
file_prefix = os.path.join(directory, "ckpt_{epoch}")

callback=[tf.keras.callbacks.ModelCheckpoint(filepath=file_prefix, save_weights_only=True)]


In [32]:
epochs=45

In [33]:

history = model.fit(dataset, epochs=epochs, steps_per_epoch=steps_per_epoch, callbacks=callback)

Epoch 1/45
158/158 [==============================] - 107s 675ms/step - loss: 2.6684
Epoch 2/45
158/158 [==============================] - 104s 656ms/step - loss: 1.9597
Epoch 3/45
158/158 [==============================] - 103s 654ms/step - loss: 1.6832
Epoch 4/45
158/158 [==============================] - 104s 657ms/step - loss: 1.5192
Epoch 5/45
158/158 [==============================] - 105s 664ms/step - loss: 1.4198
Epoch 6/45
158/158 [==============================] - 104s 659ms/step - loss: 1.3533
Epoch 7/45
158/158 [==============================] - 104s 657ms/step - loss: 1.3040
Epoch 8/45
158/158 [==============================] - 105s 662ms/step - loss: 1.2615
Epoch 9/45
158/158 [==============================] - 104s 657ms/step - loss: 1.2278
Epoch 10/45
158/158 [==============================] - 104s 659ms/step - loss: 1.1944
Epoch 11/45
158/158 [==============================] - 103s 654ms/step - loss: 1.1610
Epoch 12/45
158/158 [==============================] - 105s 663

In [34]:
tf.train.latest_checkpoint(directory)

'./checkpoints/ckpt_45'

In [35]:
model = build_model(vocabulary_size, embedding_dimension, recurrent_nn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(directory))

model.build(tf.TensorShape([1, None]))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21504     
_________________________________________________________________
unified_gru_1 (UnifiedGRU)   (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86100     
Total params: 4,045,908
Trainable params: 4,045,908
Non-trainable params: 0
_________________________________________________________________


In [37]:

def generate_text(model, start_string, temperature, characters_to_generate):

  # Vectorise  start string into numbers
    input_string = [char_to_index[char] for char in start_string]
    input_string = tf.expand_dims(input_string, 0)

  # Empty string to store  generated text
    generated = []

  # (Batch size is 1)
    model.reset_states()
    for i in range(characters_to_generate):
        predictions = model(input_string)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(logits=predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_string = tf.expand_dims([predicted_id], 0)

        generated.append(index_to_char[predicted_id])

    return (start_string + ''.join(generated)) # generated is a list

In [38]:
# In the arguments, a low temperature gives more predictable text whereas a high temperature gives more random text.
# Also this is where you can change the start string.
generated_text = generate_text(model=model, start_string="Pip", temperature=0.1, characters_to_generate = 1000)
print(generated_text)

Pip!”

“So it was.”

“Astonishing!” said Joe, in the nature of an umbrella.

“Then, as it were now as for the other convict. “The fear of our own little nor
stones of the forge. I was falling into my head to look if the way by which I had come to be a man. A deserting your mind to him, my indertrodic work in any convicts!” Then both very well. I
thought it would have been more crack with the forge and Mill Pond Bank, Clara
was not a variety of being in common black piece of paper, and put the two convicts were
bought, the more certain I am of a circle.

“Lookee here, Pip, look at his door.

In the evening there was a lady by which I was always creeping the fire at the windows of the copyright holder), the waiter reappeared.

“Why you see, old chap. They'll do you suppose the attempt to do it
distinctly thanked him and said that Mr. Pumblechook was on my shoulder by some one
of these days, and as Mr. Pumblechook was not at all likely he could
have done it. I had done it, under the silen